# Morbilidad Hospitalaria España 2016-2019
## Creación dataframe para visualización

[Datos INE](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176778&menu=resultados&secc=1254736195291&idp=1254735573175#!tabs-1254736195291)

In [1]:
import pandas as pd
import datetime
import json

In [2]:
#tenemos 4 dataframes por años desde 2016 a 2019
morbi16 = pd.read_csv('../data/morbilidad/EMH_2016_G01.csv',sep='\t')
morbi17 = pd.read_csv('../data/morbilidad/EMH_2017_G01.csv',sep='\t')
morbi18 = pd.read_csv('../data/morbilidad/EMH_2018_G01.csv',sep='\t')
morbi19 = pd.read_csv('../data/morbilidad/EMH_2019_G01.csv',sep='\t')

In [3]:
morbilidad = pd.concat([morbi16,morbi17,morbi18,morbi19])

In [4]:
morbilidad.sample(5)

,Norden,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,EdadMes,EdadDias,DiasEstancia,Factor
820863,44820815,48,2,48,2,21082017,G40109,1,91,1,3,7,1.00000
409986,35779559,38,2,38,2,2052019,C773,1,60,6,16,15,1.00977
505317,15355698,14,1,14,2,26122016,D291,1,70,11,15,12,1.00000
927739,12529869,11,2,11,1,21022018,H509,1,6,1,2,1,1.00000
509061,7822163,8,2,8,1,28052017,D259,1,42,2,11,2,1.00974


In [5]:
#el código de las enferemades mentales empeizan por F (hacemos una selección)
morbilidad = morbilidad[morbilidad["DiagPrin"].str.contains('^F')]
morbilidad.DiagPrin = morbilidad.DiagPrin.str.strip()

In [6]:
morbilidad.drop(['Norden','EdadDias','EdadMes'],axis=1,inplace = True)

In [7]:
morbilidad.DiagPrin.unique()[0:20] #extraer F + 2 primero dígitos para tener los tipos de enfermedasdes mentales agrupadas

array(['F0150', 'F0151', 'F0390', 'F0391', 'F04', 'F05', 'F060', 'F061',
       'F062', 'F0630', 'F0631', 'F0632', 'F0633', 'F0634', 'F064',
       'F068', 'F070', 'F0781', 'F0789', 'F079'], dtype=object)

In [8]:
morbilidad["code"] = morbilidad.DiagPrin.str.extract(r'(^F..)')

In [9]:
morbilidad.code.unique()

array(['F01', 'F03', 'F04', 'F05', 'F06', 'F07', 'F09', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29', 'F30', 'F31',
       'F32', 'F33', 'F34', 'F39', 'F40', 'F41', 'F42', 'F43', 'F44',
       'F45', 'F48', 'F50', 'F51', 'F52', 'F53', 'F55', 'F59', 'F60',
       'F63', 'F64', 'F65', 'F66', 'F68', 'F69', 'F70', 'F71', 'F72',
       'F73', 'F78', 'F79', 'F80', 'F81', 'F82', 'F84', 'F88', 'F89',
       'F90', 'F91', 'F93', 'F94', 'F95', 'F98', 'F99'], dtype=object)

## Añadimos códigos CIE (Capítulos y diagnósticos)

In [10]:
#importamos la tabla de correspondencia entre código de diagnóstico agrupadas y nombre
diagnos = pd.read_csv("../data/diagnos.csv")

In [11]:
diagnos.sample(3) 

,code,diag,cap,title
9,F09,Trastornos mentales orgánicos o sintomáticos s...,F00-F09,"Trastornos mentales orgánicos, incluidos los t..."
23,F23,Trastornos psicóticos agudos y transitorios,F20-29,"Esquizofrenia, trastornos esquizotípicos y tra..."
18,F18,Trastornos mentales y de comportamiento debido...,F10-F19,Trastornos mentales y de comportamiento debido...


In [12]:
diagnos.code.unique()

array(['F00', 'F01', 'F02', 'F03', 'F04', 'F05', 'F06', 'F07', 'F08',
       'F09', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17',
       'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28',
       'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F39', 'F40', 'F41',
       'F42', 'F43', 'F44', 'F45', 'F48', 'F50', 'F51', 'F52', 'F53',
       'F54', 'F55', 'F59', 'F60', 'F62', 'F63', 'F64', 'F66', 'F68',
       'F69', 'F65', 'F70', 'F71', 'F72', 'F73', 'F78', 'F79', 'F80',
       'F81', 'F82', 'F83', 'F84', 'F88', 'F89', 'F90', 'F91', 'F92',
       'F93', 'F94', 'F95', 'F98', 'F99'], dtype=object)

In [13]:
dicc_diag = {} #crear diccionario con diagnósticos y códigos
for i,r in diagnos.iterrows():
    dicc_diag[r.code] = r.diag

In [14]:
dicc_cap = {} #crear diccionario con diagnósticos y códigos
for i,r in diagnos.iterrows():
    dicc_cap[r.code] = r.cap

In [15]:
morbilidad["cap"] = morbilidad.code.map(dicc_cap) #creamos nueva columnas

In [16]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,code,cap
749471,8,1,8,1,27062019,F250,1,44,75,1.04887,F25,F20-29
707777,32,1,32,2,14032016,F250,1,39,26,1.00000,F25,F20-29
722983,28,1,31,1,23032019,F1420,1,51,1,1.01201,F14,F10-F19


In [17]:
discapacidad = morbilidad[morbilidad["cap"] == 'F70-79'].index #dropeamos el tipo de retraso mental
morbilidad.drop(discapacidad,inplace = True)

## Creamos fecha de ingreso

In [18]:
#creamos fecha a través de FxAlta
morbilidad.FxAlta = morbilidad.FxAlta.astype('string')

In [19]:
morbilidad['fecha_al'] = pd.to_datetime({'year': morbilidad.FxAlta.str[-4:],
                           'month': morbilidad.FxAlta.str[-6:-4],
                           'day': morbilidad.FxAlta.str[:-6]
                           })

In [20]:
#creamos fecha ingreso restando a fecha_al el número de día ingresado (DiasEstancia)
morbilidad['fecha_ing'] = morbilidad.apply(lambda fila: fila['fecha_al'] - datetime.timedelta(days = fila['DiasEstancia']), axis= 1)

In [21]:
morbilidad.drop(['FxAlta'],axis=1,inplace = True)

In [22]:
# Cambiamos algunos datos categóricos que vienen como números 

In [23]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,code,cap,fecha_al,fecha_ing
737068,8,2,8,2,F333,1,84,19,1.02915,F33,F30-39,2016-06-01,2016-05-13
727372,33,2,33,2,F23,1,50,17,1.00000,F23,F20-29,2018-10-11,2018-09-24
743569,35,1,35,2,F3113,1,66,47,1.06431,F31,F30-39,2018-05-09,2018-03-23


In [24]:
#diccionarios códigos
dic_sexo = {1:'Hombre',2:'Mujer'}
dic_entrada = {1:'Ordinario',2:'Urgente'}
dic_alta = {1: "Curación",2:'Traslado',3:'Fallecimiento',4:'Otras'}
num_str = {'1':'01',  '2':'02',  '3':'03',  '4':'04',  '5':'05',  '6':'06',  '7':'07',  '8':'08',  '9':'09'}

In [25]:
morbilidad.ProvHosp = morbilidad.ProvHosp.astype('string').replace(num_str)
morbilidad.ProvResi = morbilidad.ProvResi.astype('string').replace(num_str)

In [26]:
morbilidad['Entrada'] = morbilidad.DiagEntr.map(dic_entrada)
morbilidad['Alta'] = morbilidad.MotivoAlta.map(dic_alta)
morbilidad['SexNom'] = morbilidad.Sexo.map(dic_sexo)

In [27]:
#preparar provincias para geojsaon (nombre)

In [28]:
#diccionario
geo_json = r"../data/socioecon/spain_provinces.geojson"
with open (geo_json) as geo_file:
    provincias = json.load(geo_file)

cod_prov = {}
for i in range(52):    
    cod_prov[provincias['features'][i]['properties']['cod_prov']] = provincias['features'][i]['properties']['name']

In [29]:
morbilidad['ProvHosp_nom'] = morbilidad.ProvHosp.map(cod_prov)
morbilidad['ProvResi_nom'] = morbilidad.ProvResi.map(cod_prov)

In [30]:
#prov resi 53 = extranjeros
morbilidad.ProvResi_nom.fillna('Extranjeros',inplace = True)

In [31]:
morbilidad.isna().sum()

ProvHosp        0
Sexo            0
ProvResi        0
DiagEntr        0
DiagPrin        0
MotivoAlta      0
EdadAnios       0
DiasEstancia    0
Factor          0
code            0
cap             0
fecha_al        0
fecha_ing       0
Entrada         0
Alta            0
SexNom          0
ProvHosp_nom    0
ProvResi_nom    0
dtype: int64

In [34]:
morbilidad.columns = ['ProvHosp', 'Sexo', 'ProvResi', 'DiagEntr', 'DiagPrin', 'MotivoAlta',
       'Edad', 'Dias', 'Factor', 'code', 'cap', 'fecha_al',
       'fecha_ing', 'Entrada', 'Alta', 'SexNom', 'ProvHosp_nom',
       'ProvResi_nom']

In [35]:
morbilidad.to_csv("../data/morbilidad_Fgen.csv",index=False)